In [1]:
! pip install -r ../requirements.txt


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np
import torch
import copy
import math

import sys
import os
sys.path.append(os.path.expanduser("~/repos/NeurOps/pytorch"))
from neurops import *

from growprune import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/kaitlin/repos/NeurOps/pytorch/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kaitlin/repos/NeurOps/pytorch/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
hidden = [16]
regression = False

base_data = ToyData(64, 32, 5000, regression=regression)
train_loader, val_loader, test_loader = split_dataset(base_data, val_size = 0.1, test_size = 0.1, batch_size = 128)



model = ModSequential(
        ModLinear(64, hidden[0], masked=True, prebatchnorm=True, learnable_mask=True),
        #ModLinear(hidden[0], hidden[1], masked=True, prebatchnorm=True, learnable_mask=True),
        ModLinear(hidden[0], 1 if regression else 2, masked=True, prebatchnorm=True),
        track_activations=True,
        track_auxiliary_gradients=True
    ).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss() if regression else torch.nn.CrossEntropyLoss()

print("This model has {} effective parameters.".format(model.parameter_count(masked = True)))
#print("The conversion factor of this model is {} after layer {}.".format(model.conversion_factor, model.conversion_layer))

This model has 1090 effective parameters.


In [4]:
train(model, train_loader, optimizer, criterion, epochs=50, val_loader=val_loader, device=device, regression=regression, verbose=False)

Validation: Average loss: 0.0062, Accuracy: 232/450 (51.56%)
Validation: Average loss: 0.0060, Accuracy: 263/450 (58.44%)
Validation: Average loss: 0.0056, Accuracy: 286/450 (63.56%)
Validation: Average loss: 0.0053, Accuracy: 316/450 (70.22%)
Validation: Average loss: 0.0050, Accuracy: 333/450 (74.00%)
Validation: Average loss: 0.0047, Accuracy: 349/450 (77.56%)
Validation: Average loss: 0.0044, Accuracy: 361/450 (80.22%)
Validation: Average loss: 0.0040, Accuracy: 376/450 (83.56%)
Validation: Average loss: 0.0037, Accuracy: 384/450 (85.33%)
Validation: Average loss: 0.0034, Accuracy: 397/450 (88.22%)
Validation: Average loss: 0.0030, Accuracy: 400/450 (88.89%)
Validation: Average loss: 0.0029, Accuracy: 406/450 (90.22%)
Validation: Average loss: 0.0026, Accuracy: 409/450 (90.89%)
Validation: Average loss: 0.0025, Accuracy: 414/450 (92.00%)
Validation: Average loss: 0.0022, Accuracy: 413/450 (91.78%)
Validation: Average loss: 0.0021, Accuracy: 416/450 (92.44%)
Validation: Average loss

In [5]:
#modded_data = copy.deepcopy(base_data)
#modded_data.shift_distribution(-1, 100)
#modded_train_loader, modded_val_loader, modded_test_loader = split_dataset(modded_data, val_size = 0.1, test_size = 0.1, batch_size = 128)

In [11]:
for power in range(int(math.log(base_data.input_dim, 2))):
    mod = copy.deepcopy(model)
    mod_optimizer = torch.optim.SGD(mod.parameters(), lr=0.01)
    mod_optimizer.load_state_dict(optimizer.state_dict())

    shift = 2**(int(math.log(base_data.input_dim, 2))-power-1)
    datasets = [val_loader]
    print("Training {} epochs per shift, shifting window by {} indices {} times".format(64//int(base_data.input_dim/shift), shift, int(base_data.input_dim/shift)))
    for step in range(int(base_data.input_dim/shift)):
        modded_data = copy.deepcopy(base_data)
        modded_data.shift_window((step+1)*shift)
        print(modded_data.window_index, end=", ")
        modded_train_loader, modded_val_loader, modded_test_loader = split_dataset(modded_data, val_size = 0.1, test_size = 0.1, batch_size = 128)
        datasets.append(modded_val_loader)
        train(mod, modded_train_loader, mod_optimizer, criterion, epochs=64//int(base_data.input_dim/shift), val_loader=modded_val_loader, device=device, regression=regression, verbose=False, val_verbose=False)
    print()
    for vl in datasets:
        test(mod, vl, criterion, device=device, regression=regression, metrics=True)
        print(", ", end="")
        for key, value in mod.activations.items():
            print("{}: {:.3f} {}".format(key, effective_rank(value, partial=True).item(), round(orthogonality_gap(value).item(), 1)), end="\t")
        print()

Training 32 epochs per shift, shifting window by 32 indices 2 times
32, 64, 
Average loss: 0.0008, Accuracy: 444/450 (98.67%), -1: 148.015 10166317.0	0: 28.277 43937.9	1: 8.515 1347.2	
Average loss: 0.0189, Accuracy: 221/450 (49.11%), -1: 148.130 10210169.0	0: 28.004 50654.9	1: 8.138 10382.8	
Average loss: 0.0008, Accuracy: 441/450 (98.00%), -1: 148.852 10539375.0	0: 27.776 39788.4	1: 10.518 3725.8	
Training 16 epochs per shift, shifting window by 16 indices 4 times
16, 32, 48, 64, 
Average loss: 0.0015, Accuracy: 429/450 (95.33%), -1: 148.714 10591118.0	0: 28.004 50568.9	1: 10.826 6668.9	
Average loss: 0.0069, Accuracy: 318/450 (70.67%), -1: 147.760 10009573.0	0: 27.281 30148.9	1: 7.761 999.8	
Average loss: 0.0136, Accuracy: 224/450 (49.78%), -1: 149.591 10602417.0	0: 28.854 51539.3	1: 9.059 2616.2	
Average loss: 0.0061, Accuracy: 325/450 (72.22%), -1: 148.683 10388355.0	0: 27.377 32574.0	1: 6.402 335.0	
Average loss: 0.0014, Accuracy: 429/450 (95.33%), -1: 149.429 10691659.0	0: 27.97

In [16]:
for key, value in model.activations.items():
    print("{}: {:.3f} {}".format(key, effective_rank(value, partial=True).item(), round(orthogonality_gap(value).item(), 1)), end="\t")
print()
print(activation_variance(model.activations["0"]))

for power in range(int(math.log(base_data.input_dim, 2))):
    mod = copy.deepcopy(model)
    mod_optimizer = torch.optim.SGD(mod.parameters(), lr=0.01)
    mod_optimizer.load_state_dict(optimizer.state_dict())

    shift = 2**(int(math.log(base_data.input_dim, 2))-power-1)
    datasets = [val_loader]
    
    print("Training {} epochs per shift, shifting window by {} indices {} times".format(64//int(base_data.input_dim/shift), shift, int(base_data.input_dim/shift)))
    for step in range(int(base_data.input_dim/shift)-2):
        modded_data = copy.deepcopy(base_data)
        modded_data.shift_distribution(adder=((step+1)*shift)/32)
        modded_train_loader, modded_val_loader, modded_test_loader = split_dataset(modded_data, val_size = 0.1, test_size = 0.1, batch_size = 128)
        datasets.append(modded_val_loader)
        train(mod, modded_train_loader, mod_optimizer, criterion, epochs=64//int(base_data.input_dim/shift), val_loader=modded_val_loader, device=device, regression=regression, verbose=False, val_verbose=False)
        for key, value in mod.activations.items():
            print("{}: {:.3f} {}".format(key, effective_rank(value, partial=True).item(), round(orthogonality_gap(value).item(), 1)), end="\t")
        print()
        print(activation_variance(mod.activations["0"]))
    print()
    for vl in datasets:
        test(mod, vl, criterion, device=device, regression=regression, metrics=True)
        print(", ", end="")
        for key, value in mod.activations.items():
            print("{}: {:.3f} {}".format(key, effective_rank(value, partial=True).item(), round(orthogonality_gap(value).item(), 1)), end="\t")
        print()

-1: 147.673 10035187.0	0: 27.996 61267.8	1: 10.830 37350.9	
tensor([0.3890, 0.1954, 0.3459, 0.7973, 0.0509, 0.0724, 0.5273, 1.0188, 0.2346,
        0.2770, 0.0758, 0.5836, 0.4636, 0.2153, 0.0439, 0.3187])
Training 32 epochs per shift, shifting window by 32 indices 2 times

Average loss: 0.0006, Accuracy: 441/450 (98.00%), -1: 148.120 10245874.0	0: 27.025 29761.9	1: 14.030 76687.7	
Training 16 epochs per shift, shifting window by 16 indices 4 times
-1: 151.746 24108056.0	0: 27.996 173721.8	1: 11.771 53829.2	
tensor([0.4379, 0.2986, 0.1063, 0.9323, 0.0204, 0.0697, 0.6269, 0.0410, 0.1795,
        0.0164, 0.1723, 0.8169, 0.0486, 0.0930, 0.0265, 0.4312])
-1: 158.716 142573168.0	0: 29.119 411521.7	1: 9.506 11114.8	
tensor([3.7592e-01, 2.4798e-01, 2.3430e-01, 9.4914e-01, 3.1781e-03, 1.2304e-01,
        9.7981e-01, 2.5686e-02, 3.5922e-01, 5.7804e-04, 2.0119e-01, 1.3373e+00,
        3.6713e-02, 1.1464e-01, 1.2492e-02, 4.9028e-01])

Average loss: 0.0044, Accuracy: 317/450 (70.44%), -1: 147.414 9

In [ ]:
#track and plot neuron-wise scores across distribution shifts